# WorkFlow 1: Load WebSite content into Vector DB

1.   Use LangChain LCEL
2.  Prompting + LCEL + Output Parser
3.  RAG (build once, re-use) with sources




In [ ]:
#Map Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#%pip install --upgrade jupyter-client

In [ ]:
%pip install -qU \
    "requests" \
    "langchain" \
    "langchain-openai" \
    "langchain-community" \
    "langchain-text-splitters" \
    beautifulsoup4 lxml faiss-cpu langchainhub tavily-python "gradio"

In [ ]:
import importlib
def _ver(name):
    try:
        m = importlib.import_module(name)
        return getattr(m, "__version__", "n/a")
    except Exception as e:
        return f"not installed ({e})"
print("langchain           :", _ver("langchain"))
print("langgraph           :", _ver("langgraph"))
print("langchain-core      :", _ver("langchain_core"))
print("langchain-community :", _ver("langchain_community"))
print("langchain-openai    :", _ver("langchain_openai"))
print("langchainhub        :", _ver("langchainhub"))
print("langchain-text-splitters:", _ver("langchain_text_splitters"))
print("faiss-cpu           :", _ver("faiss"))
print("tavily-python       :", _ver("tavily"))

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Load URL content

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1) Load docs (pick any public pages you want indexed)
urls = [
    "https://www.apple.com/",
    "https://www.apple.com/iphone/",
  # "https://www.apple.com/ipad/",
  # "https://www.apple.com/watch/",
    "https://www.apple.com/mac/"
]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def load_and_summarize(url: str) -> Document:
    """
    Loads an Apple.com page, extracts text, summarizes it,
    and returns a LangChain Document safe for RAG.
    """
    loader = WebBaseLoader(url)
    docs = loader.load()

    raw_text = docs[0].page_content

    summary = llm.invoke(
        f"Summarize the key product facts from this Apple page. "
        f"Focus on specs, features, and model differences.\n\n{raw_text}"
    )

    return Document(
        page_content=summary.content,
        metadata={"source": url}
    )


In [ ]:
docs = [load_and_summarize(url) for url in urls]


In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vectorstore = FAISS.from_documents(docs, embeddings)

vectorstore.save_local("apple_products")

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vectorstore = FAISS.load_local(
    "apple_products",
    embeddings,
    allow_dangerous_deserialization=True
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
results = retriever.invoke("best MacBook for video editing")

for r in results:
    print(r.metadata["source"])
    print(r.page_content[:300], "\n")

#Embeddings
Generate Open AI embeddings, store in in-memory database and create retreiver object for similarity search

In [ ]:
from langchain_core.prompts import ChatPromptTemplate;
system_prompt = """
You are "Apple Sales Agent", an expert Apple product specialist.

Use a ReAct-style reasoning process INTERNALLY:
- Thought: your internal reasoning about what to do next.
- Action: the tool name and JSON arguments you want to call.
- Observation: the result returned by the tool.
- Answer: the final response you will give to the user.

The user must NEVER see Thought, Action, or Observation.
They ONLY see the final Answer.

Tools you can call:

- rag_product_search(query: str)
  Use this when you need detailed product information from the product knowledge base.
  It returns an array of chunks with product_id, title, content, and source.

When using rag_product_search:
- Craft a focused query that includes product family, use case, and key constraints.
- Read the returned chunks carefully and base your Answer only on reliable information.
- If information is missing or unclear, say you don’t know rather than inventing details.

Your goals:
1. Understand the customer's needs, constraints, and context.
2. Recommend the best Apple products, configurations, and accessories.
3. Explain trade-offs clearly and concisely.
4. Never fabricate specs, prices, or availability.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

#LLM Call
Query LLM using LCEL pipeline.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# 5) LCEL pipeline: {question} flows through; {context} is produced by retriever
rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 6) Try it
rag_chain.invoke("I want to buy an IPhone")

In [ ]:
def rag_product_search(query: str) -> str:
    """Retrieve product info from FAISS vector DB."""
    docs = rag_chain.invoke("I want to an Iphone")   # <-- FIXED

    out = []
    for d in docs:
        out.append({
            "product_id": d.metadata.get("product_id"),
            "title": d.metadata.get("title"),
            "content": d.page_content,
            "source": d.metadata.get("source"),
        })
    return str(out)


In [ ]:
# @title
import gradio as gr
from langchain_core.messages import AIMessage

def _final_text(res):
    if isinstance(res, AIMessage):
        return res.content or ""
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            if isinstance(m, AIMessage) or getattr(m, "type", "") == "ai":
                return getattr(m, "content", "") or ""
    return str(res)

def _to_messages(history, message):
    msgs = []
    for u, a in history:
        if u: msgs.append({"role": "user", "content": u})
        if a: msgs.append({"role": "assistant", "content": a})
    msgs.append({"role": "user", "content": message})
    return msgs

def _ensure_agent():
    global agent
    try:
        agent
        return agent
    except NameError:
        from langchain_openai import ChatOpenAI
        from langchain.agents import create_agent  # Changed import
        from langchain_core.tools import tool

        @tool
        def add(a: float, b: float) -> float:
            "Add two numbers."
            return a + b

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        agent = create_agent(llm, [add], system_prompt="You are helpful.")  # Changed parameter
        return agent

def chat_fn(message, history):
    try:
        ag = _ensure_agent()
        msgs = _to_messages(history, message)
        res = ag.invoke({"messages": msgs})
        return _final_text(res)
    except Exception as e:
        return f"Error: {e}"

try:
    demo.close()
except Exception:
    pass

with gr.Blocks() as demo:
    gr.Markdown("# Apple Sales Agent Chat")
    gr.Markdown("Ask about your KB (kb_search) or general queries. Web search only if TAVILY_API_KEY is set.")
    gr.ChatInterface(chat_fn)
    gr.Markdown('Tip: Try "Where are tracing docs?" or "Multiply 3.5 and 4."')  # Fixed quotes

demo.launch(share=False)

In [ ]:
# ---------- ReAct Agent Setup (create_agent) ----------

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
)

system_prompt = """
You are "Apple Sales Agent", an expert Apple product specialist.

Use a ReAct-style reasoning process INTERNALLY:
- Thought: your internal reasoning about what to do next.
- Action: the tool name and JSON arguments you want to call.
- Observation: the result returned by the tool.
- Answer: the final response you will give to the user.

The user must NEVER see Thought, Action, or Observation.
They ONLY see the final Answer.

Tools you can call:

- rag_product_search(query: str)
  Use this when you need detailed product information from the product knowledge base.
  It returns an array of chunks with product_id, title, content, and source.

When using rag_product_search:
- Craft a focused query that includes product family, use case, and key constraints.
- Read the returned chunks carefully and base your Answer only on reliable information.
- If information is missing or unclear, say you don’t know rather than inventing details.

Your goals:
1. Understand the customer's needs, constraints, and context.
2. Recommend the best Apple products, configurations, and accessories.
3. Explain trade-offs clearly and concisely.
4. Never fabricate specs, prices, or availability.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

tools = [rag_product_search]

agent = create_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# ---------- Public API for your app / UI ----------

def run_sales_agent(user_message: str) -> str:
    """
    Invoke the ReAct-style LangChain agent and return only the final answer.
    """
    result = agent.invoke({"input": user_message})
    # LangChain's create_agent returns a dict-like structure with "output"
    return result.get("output", str(result))


if __name__ == "__main__":
    # Quick manual test
    msg = "I need a MacBook for 4K video editing under $2500 with at least 1TB storage."
    reply = run_sales_agent(msg)
    print("Agent:", reply)